In [4]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 12 not upgraded.
Need to get 167 kB of archives.
After this operation, 558 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 unzip amd64 6.0-21ubuntu1 [167 kB]
Fetched 167 kB in 0s (591 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package unzip.
(Reading database ... 6778 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-21ubuntu1_amd64.deb ...
Unpacking unzip (6.0-21ubuntu1) ...
Setting up unzip (6.0-21ubuntu1) ...


In [5]:
!unzip whole_corpus_eng.zip

Archive:  whole_corpus_eng.zip
   creating: whole_corpus_eng/
   creating: whole_corpus_eng/extrapolate/
  inflating: whole_corpus_eng/.DS_Store  
  inflating: __MACOSX/whole_corpus_eng/._.DS_Store  
   creating: whole_corpus_eng/interpolate/
   creating: whole_corpus_eng/train/
  inflating: whole_corpus_eng/extrapolate/algebra__polynomial_roots_big.txt  
  inflating: __MACOSX/whole_corpus_eng/extrapolate/._algebra__polynomial_roots_big.txt  
  inflating: whole_corpus_eng/extrapolate/measurement__conversion.txt  
  inflating: __MACOSX/whole_corpus_eng/extrapolate/._measurement__conversion.txt  
  inflating: whole_corpus_eng/extrapolate/numbers__round_number_big.txt  
  inflating: __MACOSX/whole_corpus_eng/extrapolate/._numbers__round_number_big.txt  
  inflating: whole_corpus_eng/extrapolate/arithmetic__mul_big.txt  
  inflating: __MACOSX/whole_corpus_eng/extrapolate/._arithmetic__mul_big.txt  
  inflating: whole_corpus_eng/extrapolate/numbers__place_value_big.txt  
  inflating: __MACO

In [6]:
!pip install transformers

     |████████████████████████████████| 635 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 20.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 676 kB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.8 MB/s  eta 0:00:01
     |████████████████████████████████| 294 kB 25.8 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=4a34b73b47147ff5ef45d5d5dc4ec001af1b1655c24d361950e57ac52aeb29ac
  Stored in directory: /root/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses


In [1]:
import os
import re
import numpy as np
from transformers.modeling_auto import AutoModel
import torch
from time import time
import gc
from transformers import BertTokenizer, AutoTokenizer, AutoModelWithLMHead


In [2]:
!rm -rf ~/.local/share/Trash/*

In [4]:
train_files = ['whole_corpus_eng/train/'+file for file in os.listdir('whole_corpus_eng/train/') if file.endswith('.txt')]

In [5]:
interpolate_files = ['whole_corpus_eng/interpolate/'+file for file in os.listdir('whole_corpus_eng/interpolate/') if file.endswith('.txt')]
extrapolate_files = ['whole_corpus_eng/extrapolate/'+file for file in os.listdir('whole_corpus_eng/extrapolate/') if file.endswith('.txt')]

In [8]:
data_train = open('regression.tsv', 'w')

for file in train_files:
    text = open(file).read().splitlines()
    questions, answers = text[::2], text[1::2]
    qa = [(q, a) for q, a in zip(questions, answers) if a.isdigit()]
    np.random.shuffle(qa)
    
    for q, a in qa:
        data_train.write('\t'.join([q, a]) + '\n')
data_train.close()

random_train = open('random_regression.tsv', 'w')
corpus = [line for line in open('regression.tsv')]
np.random.shuffle(corpus)
for c in corpus:
    random_train.write(c)
random_train.close()

data_interpolate = open('inter_regression.tsv', 'w')

for file in interpolate_files:
    text = open(file).read().splitlines()
    questions, answers = text[::2], text[1::2]
    for q, a in zip(questions, answers):
        if a.isdigit():
            data_interpolate.write('\t'.join([q, a]) + '\n')

data_interpolate.close()

data_extra = open('extra_regression.tsv', 'w')

for file in extrapolate_files:
    text = open(file).read().splitlines()
    questions, answers = text[::2], text[1::2]
    for q, a in zip(questions, answers):
        if a.isdigit():
            data_extra.write('\t'.join([q, a]) + '\n')

data_extra.close()

In [5]:
# # Rubert
# tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
# model_bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").to(torch.device('cuda'))

In [3]:
# # Eng Bert
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model_bert = AutoModel.from_pretrained('bert-base-cased')
# # model = torch.load('en_bert_5epoch').to(torch.device('cuda'))

In [4]:
# # Multilingual Bert
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model_bert = AutoModel.from_pretrained('bert-base-multilingual-cased').to(torch.device('cuda'))

In [16]:
def gen_batches_choice_torch(file, batch_size=32):
    
    batch_q = []
    batch_a = []
    
    while True:
        for i, line in enumerate(open(file)):
            q,a = line.strip('\n').split('\t')
            batch_q.append(q)
            batch_a.append(int(a))

            if len(batch_q) == batch_size:
                yield (batch_q,
                       np.array(batch_a, dtype=np.float32))
                batch_q = []
                batch_a = []

                
def gen_batches_choice_val(file, batch_size=32):
    batch_q = []
    batch_a = []
    
    for i, line in enumerate(open(file)):

        q,a = line.strip('\n').split('\t')
        batch_q.append(q)
        batch_a.append(int(a))

        if len(batch_q) == batch_size:
            yield (batch_q,
                   np.array(batch_a, dtype=np.float32))
            batch_q = []
            batch_a = []

In [11]:
import torch.nn as nn

class Model_Reg(nn.Module):
    
    def __init__(self):

        super().__init__()          
        
        self.tokenizer = tokenizer
        self.bert = model_bert
        self.num_iter = 0
        self.fake_epoch = 0
        self.fc = nn.Linear(768, 1)
        
    def forward(self, texts): 
        texts_ids = [torch.tensor(self.tokenizer.encode(t, add_special_tokens=True)) for t in texts]
        texts_ids = torch.nn.utils.rnn.pad_sequence(texts_ids, batch_first=True).to(torch.device('cuda'))
        mask = (texts_ids != tokenizer.pad_token_id).long()
        hidden, pooler = self.bert(texts_ids, attention_mask=mask)

#         num =self.fc(pooler)
        num =self.fc(hidden[:, 0])

        
        self.num_iter += 1
        return num

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
model = Model_Reg()

In [19]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

#define metric
def accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = preds.round()
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [20]:
def train(model, iterator, batch_size, epoch):
    epoch_loss = []
    epoch_acc = []

    model.train()  

    for i, batch in enumerate(iterator):

        #resets the gradients after every batch
        optimizer.zero_grad()   

        #retrieve text and no. of words
        texts, labels = batch[0], batch[1]   

        #convert to 1D tensor
        predictions = model(texts).squeeze()

        #compute the loss
        loss = criterion(predictions, torch.log1p(torch.Tensor(labels)).to(device))        

        #compute the binary accuracy
        acc = accuracy(torch.expm1(predictions), torch.Tensor(labels).to(device))   

        #backpropage the loss and compute the gradients
        loss.backward()       

        #update the weights
        optimizer.step()      

        #loss and accuracy
        epoch_loss.append(loss.item())  
        epoch_acc.append(acc.item())
        
        if not ((model.num_iter+1)*batch_size) % 30000:
            print(model.fake_epoch)
            model.fake_epoch += 1
            print(np.mean(epoch_loss), np.mean(epoch_acc))
            epoch_loss = []
            epoch_acc = []

        if ((model.num_iter*batch_size)//3754297) >= epoch:
            return (np.mean(epoch_loss), np.mean(epoch_acc))

In [21]:
def validate(model, iterator):

#set the model in training phase
    model.train(mode=False)  
    val_loss = []
    val_acc = []
    for i, batch in enumerate(iterator):
        #retrieve text and no. of words
        texts, labels = batch[0], batch[1]   

        #convert to 1D tensor
        predictions = model(texts).squeeze()  

        #compute the loss
        loss = criterion(predictions, torch.log1p(torch.Tensor(labels)).to(device))        

        #compute the binary accuracy
        acc = accuracy(torch.expm1(predictions), torch.Tensor(labels).to(device))

        val_loss.append(loss.item())  
        val_acc.append(acc.item())  

    return (np.mean(val_loss), np.mean(val_acc))

In [24]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from time import time

for i in range(0, 21):
    start = time()
    iterator = gen_batches_choice_torch('random_regression.tsv', 16)
    l, a = train(model, iterator, 16, i)
    print(f'On epoch: {i}; Train Loss: {l}; Train Accuracy: {a}')
    iterator = gen_batches_choice_val('inter_regression.tsv',16)
    li, ai = validate(model, iterator)
    iterator = gen_batches_choice_val('extra_regression.tsv',16)
    le, ae = validate(model, iterator)
    print(f'On epoch: {i}; Inter Loss: {li}; Inter Accuracy: {ai}')
    print(f'On epoch: {i}; Extra Loss: {le}; Extra Accuracy: {ae}')
    print(f'Elapsed Time: {time()-start}')
    gc.collect()
    torch.cuda.empty_cache()

On epoch: 0; Train Loss: 10.592344284057617; Train Accuracy: 0.125
On epoch: 0; Inter Loss: 13.918464422303495; Inter Accuracy: 0.03295750216825672
On epoch: 0; Extra Loss: 52.60209835153453; Extra Accuracy: 0.029846938775510205
Elapsed Time: 48.677154302597046
0
9.18378685022211 0.03558781869688385
1
6.2876959176540375 0.04586666666666667
2
3.800500013868014 0.0558


In [20]:
TRAIN_LEN = 0
INTER_LEN = 0
EXTRA_LEN = 0
unique = set()
for line in open('random_regression.tsv'):
    q,a = line.strip('\n').split('\t')
    TRAIN_LEN += 1
    
for line in open('inter_regression.tsv'):
    q,a = line.strip('\n').split('\t')
    INTER_LEN += 1
    
    
for line in open('extra_regression.tsv'):
    q,a = line.strip('\n').split('\t')
    unique.add(a)
    EXTRA_LEN += 1
    
# for line in open('extrapolate.tsv'):
#     EXTRA_LEN += 1


In [21]:
TRAIN_LEN

3754297

In [22]:
INTER_LEN

18452

In [23]:
EXTRA_LEN

3933